In [59]:
from langgraph.graph import StateGraph, START,END
from langchain_ollama import ChatOllama
from typing import TypedDict, Annotated
from pydantic import BaseModel, Field
import operator

In [60]:
model =  ChatOllama(model="initium/law_model:latest")

In [61]:
class evaluation_schema(BaseModel):
    feedback : str = Field(description = "Detailed feedback for the essay")
    score: int = Field(description = 'Score out of 10', ge=10,le=10)

In [62]:
structured_model = model.with_structured_output(evaluation_schema)

In [63]:
essay = """My Day in City

Yesterday I going to city with my friend for buy some thing. We wake up very late so no time for eat breakfast, only drink tea. After we catch bus but bus is very much crowded and noisy and smell not good. My friend he say we should go walking but I tell no because my leg already pain from yesterday football play.

In city we see many people is selling all kind thing, like fruit, shoe, phone and many animal also. I want to buy mango but seller tell me price too much high so I angry and walk away. Then we going to shop for buy bag, but bag is big and heavy and color not so nice.

We eat lunch in small hotel, food is not so taste good and plate also not clean but we still eat because we hungry too much. Then raining come and we no have umbrella so we become all wet. My shoe making sound chap-chap when walk.

When going back home, bus not coming for long time so we sit on bench and wait. Finally bus come but driver very fast drive and many time we almost fall down. At home I feeling very much tired and my clothes full dirty. It was good and bad day both.

"""

In [64]:
prompt = f"Evaluate the language quality of the following essay and provide a feedback and assign a score out of 10 \n {essay}"

In [ ]:
result = structured_model.invoke(prompt)

In [ ]:
print(result.score)

10


In [ ]:
print(result.feedback)

The essay provides a comprehensive overview of India's strengths, challenges, and opportunities in the age of artificial intelligence (AI). The author demonstrates their ability to organize information coherently and provide context to key concepts. However, the writing could benefit from more concrete examples and statistics to support the claims made. For instance, while the essay states that India has a burgeoning base of AI researchers, it does not mention how many AI researchers are present in the country or the number of institutions involved in AI research.


In [ ]:
result

evaluation_schema(feedback="The essay provides a comprehensive overview of India's strengths, challenges, and opportunities in the age of artificial intelligence (AI). The author demonstrates their ability to organize information coherently and provide context to key concepts. However, the writing could benefit from more concrete examples and statistics to support the claims made. For instance, while the essay states that India has a burgeoning base of AI researchers, it does not mention how many AI researchers are present in the country or the number of institutions involved in AI research.", score=10)

In [ ]:
class UPSCState(TypedDict):
    essay : str

    language_feedback : str
    analysis_feedback : str
    clarity_feedback : str
    overall_feedback : str

    individual_scores : Annotated[list[int] , operator.add]

    avg_score : float


In [ ]:
def evaluate_language(state: UPSCState):
    prompt = f"""You are strict auditor of UPSC. You have to evaluate the language of the candidate. The candidate has given the answer.
    Evalaute the language quality of the following essay and be a strict assessor and provide a feedback and assign a score out of 10 \n {state['essay']}.
"""
    
    output = structured_model.invoke(prompt)

    return {'language_feedback':output.feedback, 'individual_scores':[output.score]}

In [ ]:
def evaluate_analysis(state: UPSCState):
    prompt = f"""You are strict auditor of UPSC. You have to evaluate the language of the candidate. The candidate has given the answer.
    Evalaute the depth quality of analysis of the following essay and be a strict assessor and provide a feedback and assign a score out of 10 \n {state['essay']}.
"""
    
    output = structured_model.invoke(prompt)

    return {'analysis_feedback':output.feedback, 'individual_scores':[output.score]}

In [ ]:
def evaluate_thought(state: UPSCState):
    prompt = f"""You are strict auditor of UPSC. You have to evaluate the language of the candidate. The candidate has given the answer.
    Evalaute the clarity of thought of the following essay as a strict assessor and provide a feedback and assign a score out of 10 \n {state['essay']}.
"""
    
    output = structured_model.invoke(prompt)

    return {'clarity_feedback':output.feedback, 'individual_scores':[output.score]}

In [ ]:
def final_evaluation(state : UPSCState):
    # summary feedback
    prompt = f"""Based on the following feedback create a summarized feedback \n language feedaback  - {state['language_feedback']} \n depth of analysis feedback -{state['analysis_feedback']} \n clarity of thought feedback - {state['clarity_feedback']} """

    final_feedback = model.invoke(prompt)

    # avg score

    avg_score  = sum(state['individual_scores'])/len(state['individual_scores'])

    return {'overall_feedback':final_feedback, 'avg_score':avg_score}



In [ ]:
graph = StateGraph(UPSCState)

graph.add_node('evaluate_language', evaluate_language)
graph.add_node('evaluate_analysis', evaluate_analysis)
graph.add_node('evaluate_thought', evaluate_thought)
graph.add_node('final_evaluation', final_evaluation)

# edges

graph.add_edge(START,'evaluate_language')
graph.add_edge(START,'evaluate_analysis')
graph.add_edge(START,'evaluate_thought')

graph.add_edge('evaluate_language','final_evaluation')
graph.add_edge('evaluate_analysis','final_evaluation')
graph.add_edge('evaluate_thought','final_evaluation')

graph.add_edge('final_evaluation',END)

workflow = graph.compile()

In [ ]:
initial_state = {'essay': essay}

In [ ]:
final_state = workflow.invoke(initial_state)

In [ ]:
final_state

{'essay': "India in the Age of AI.\nAs the world enters a transformative era defined by artificial intelligence (AI), India stands at a critical juncture — one where it can either emerge as a global leader in AI innovation or risk falling behind in the technology race. The age of AI brings with it immense promise as well as unprecedented challenges, and how India navigates this landscape will shape its socio-economic and geopolitical future.\n\nIndia's strengths in the AI domain are rooted in its vast pool of skilled engineers, a thriving IT industry, and a growing startup ecosystem. With over 5 million STEM graduates annually and a burgeoning base of AI researchers, India possesses the intellectual capital required to build cutting-edge AI systems. Institutions like IITs, IIITs, and IISc have begun fostering AI research, while private players such as TCS, Infosys, and Wipro are integrating AI into their global services. In 2020, the government launched the National AI Strategy (AI for